#### Download Requested Data from BigQuery

In [40]:
import pandas as pd
import numpy as np
import os
import urllib.request

In [53]:
# 차종 종류 파악
car_info_types = pd.read_csv('./car_info_types_v2.csv')


In [59]:
'''
input; target_class_id (int)
output; query (string)
'''

def create_query(target_class_id):

    query = f"""
        WITH car_info AS (
            SELECT *
            FROM `socar-data.tianjin_replica.car_info`
            WHERE (class_id = {target_class_id})
        )

        SELECT
        csi.*,
        CONCAT(csi.upload_host, csi.filename) AS image_url, 
        car_info.class_id,
        car_info.car_name

        FROM `socar-data.tianjin_replica.car_cs_info` AS csi

        LEFT JOIN car_info
        ON csi.car_id = car_info.id

        WHERE car_name is not null
        
        LIMIT 1500
    """
    
    return query

In [63]:
save_base = '../dataset/'

nums = list()

for class_id in car_info_types['class_id'].values:
    
    class_id = class_id
    car_name = car_info_types[car_info_types['class_id']==class_id]['car_name'].values[0]
    num_data = car_info_types[car_info_types['class_id']==class_id]['number_at_1000'].values[0]
    
    if num_data == 1000:
        query = create_query(class_id)

        # download the dataset with query
        df = pd.read_gbq(query=query, project_id='socar-data', dialect='standard')
        
        filenames = df['image_url'].values
        
        save_path = os.path.join(save_base, car_name)
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        
        for filename in filenames:
            try:
                save_filepath = os.path.join(save_path, filename.split('/')[-1])
                urllib.request.urlretrieve(filename, save_filepath)
                
            except Exception as e:
                pass
            
    else:
        pass

Downloading: 100%|██████████| 1500/1500 [00:00<00:00, 2870.03rows/s]


KeyboardInterrupt: 